In [ ]:
# Third-party
from astropy.constants import c
import astropy.coordinates as coord
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

from astropy.tests.helper import quantity_allclose

In [ ]:
ra = np.random.uniform(100, 200, size=10000) * u.deg
dec = np.random.uniform(-30, 30, size=ra.size) * u.deg
distance = np.random.uniform(10, 100, size=ra.size) * u.pc

In [ ]:
icrs = coord.ICRS(ra=ra, dec=dec, distance=distance)

In [ ]:
%timeit icrs.distance.to(u.arcsecond, u.parallax()).value
%timeit icrs.distance.to_value(u.arcsecond, u.parallax())

In [ ]:
1 / np.sqrt(1 - (100*u.pc/u.Myr / c)**2)

In [ ]:
rep = coord.CartesianRepresentation(x=[-10., 10.]*u.pc, y=0*u.pc, z=0*u.pc)
dif = coord.CartesianDifferential(d_x=0*u.pc/u.Myr, 
                                  d_y=[100.,100]*u.pc/u.Myr, 
                                  d_z=0*u.pc/u.Myr)
rep = rep.with_differentials(dif)
c1 = coord.SkyCoord(rep)
c1 = c1[1]
c1

In [ ]:
c2 = c1.apply_space_motion(dt=10*u.yr)
c2.velocity - c1.velocity

In [ ]:
# assert quantity_allclose(c2.cartesian.xyz, [10., 1E-3, 0.]*u.pc, atol=1E-6*u.pc)
# assert quantity_allclose(c2.cartesian.differentials['s'].d_xyz, dif.d_xyz, atol=1*u.cm/u.s)

---

### Directly call ERFA

In [ ]:
from astropy._erfa import starpv, pvstar

In [ ]:
dt = (10*u.year).to(u.day).value

In [ ]:
rep = coord.CartesianRepresentation(x=10.*u.pc, y=0*u.pc, z=0*u.pc)
dif = coord.CartesianDifferential(d_x=0*u.pc/u.Myr, 
                                  d_y=100.*u.pc/u.Myr, 
                                  d_z=0*u.pc/u.Myr)
rep = rep.with_differentials(dif)
c1 = coord.SkyCoord(rep)
c1.set_representation_cls(s=coord.SphericalDifferential)
c1

In [ ]:
pv1 = starpv(c1.ra.radian, c1.dec.radian, 
             c1.pm_ra.to(u.radian/u.yr), 
             c1.pm_dec.to(u.radian/u.yr), 
             c1.distance.parallax.to(u.arcsec).value, 
             c1.radial_velocity.to(u.km/u.s).value)
(pv1[0]*u.au).to(u.pc), (pv1[1]*u.au/u.day).to(u.pc/u.Myr)

In [ ]:
ERFA_DC = c.to(u.au/u.day).value
tl1 = np.linalg.norm(pv1[0]) / ERFA_DC
tl1

In [ ]:
pv = pv1.copy()
pv[0] = pv1[0] + pv1[1] * (dt + tl1)

In [ ]:
r2 = pv[0].dot(pv[0])
rdv = pv[0].dot(pv[1])
v2 = pv[1].dot(pv[1])
c2mv2 = ERFA_DC*ERFA_DC - v2
tl2 = (-rdv + np.sqrt(rdv*rdv + c2mv2*r2)) / c2mv2
tl2

In [ ]:
pv2 = pv1.copy()
pv2[0] = pv2[0] + pv2[1] * (dt + (tl1 - tl2))

In [ ]:
pv2

In [ ]:
rep_ = coord.CartesianRepresentation(pv2[0]*u.au)
dif_ = coord.CartesianDifferential(pv2[1]*u.au/u.day)
rep_ = rep_.with_differentials(dif_)
rep_ = rep_.represent_as(coord.SphericalRepresentation, 
                         coord.SphericalDifferential)
star2_rep = [rep_.lon.radian, rep_.lat.radian, 
             rep_.differentials['s'].d_lon.to(u.radian/u.yr).value,
             rep_.differentials['s'].d_lat.to(u.radian/u.yr).value,
             rep_.distance.parallax.to(u.arcsec).value,
             rep_.differentials['s'].d_distance.to(u.km/u.s).value]
star2_rep = np.array(star2_rep)
star2_rep

In [ ]:
star2 = np.array(pvstar(pv2.copy()))
print(star2)

In [ ]:
star2 - star2_rep

In [ ]:
c2_2 = coord.ICRS(ra=star2[0]*u.rad, dec=star2[1]*u.rad,
                  pm_ra=star2[2]*u.rad/u.yr, pm_dec=star2[3]*u.rad/u.yr,
                  distance=coord.Distance(parallax=star2[4]*u.arcsec),
                  radial_velocity=star2[5]*u.km/u.s,
                  differential_cls=coord.SphericalDifferential)

In [ ]:
c2_2.cartesian.differentials['s']